## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../weather_database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,77.99,85,80,16.42,light rain
1,1,Hilo,US,19.7297,-155.0900,81.28,93,100,6.91,light rain
2,2,Albany,US,42.6001,-73.9662,55.81,62,100,4.74,overcast clouds
3,3,Upernavik,GL,72.7868,-56.1549,2.55,94,58,4.90,broken clouds
4,4,Mataura,NZ,-46.1927,168.8643,64.24,61,100,2.30,overcast clouds


In [3]:
#In Step 2, write two input statements that prompt the user to enter their 
#minimum and maximum temperature criteria for their vacation.
#-----------
# 2. Prompt the user to enter minimum and maximum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 90


In [4]:
#In Step 3, use the loc method to filter the city_data_df DataFrame for 
#temperature criteria collected in Step 2, then create a new DataFrame.
#-------------------
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
matching_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
matching_cities_df.count()

City_ID                325
City                   325
Country                325
Lat                    325
Lng                    325
Max Temp               325
Humidity               325
Cloudiness             325
Wind Speed             325
Current Description    325
dtype: int64

In [5]:
# 4a. Determine if there are any empty rows.
# We drop NAs in the previous cell, so there will be no null values
matching_cities_df.dropna()
print(matching_cities_df.count())

City_ID                325
City                   325
Country                325
Lat                    325
Lng                    325
Max Temp               325
Humidity               325
Cloudiness             325
Wind Speed             325
Current Description    325
dtype: int64


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
#No need to execute this step. DataFrame is already clean

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = matching_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,77.99,light rain,-23.1203,-134.9692,
1,Hilo,US,81.28,light rain,19.7297,-155.0900,
7,Sao Filipe,CV,71.76,few clouds,14.8961,-24.4956,
10,Alofi,NU,84.09,scattered clouds,-19.0595,-169.9187,
17,Port Alfred,ZA,68.79,clear sky,-33.5906,26.8910,
18,Luang Prabang,LA,76.96,broken clouds,19.8856,102.1347,
20,Saposoa,PE,69.93,broken clouds,-6.9340,-76.7716,
22,Atuona,PF,79.07,overcast clouds,-9.8000,-139.0333,
23,Senanga,ZM,65.12,broken clouds,-16.1167,23.2667,
28,San Cristobal,VE,73.58,overcast clouds,7.7669,-72.2250,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
    # 6c. Get latitude and longitude from DataFrame
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    # 6e. Make request and retrieve the JSON data from the search. 
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        


    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.

print(f"Before cleaning empty hotel name: {hotel_df[hotel_df.columns[0]].count()}")

for index, row in hotel_df.iterrows():
    city_name = row["City"]
    hotel_name = row["Hotel Name"]
    if hotel_name == "" or len(hotel_name) == 0:
        #print(f"Hotel for city {city_name} is empty: <{hotel_name}>")
        hotel_df.loc[hotel_df["City"]==city_name, "Hotel Name"] = np.nan

clean_hotel_df = hotel_df.dropna()
print(f"Before cleaning empty hotel name: {clean_hotel_df[clean_hotel_df.columns[0]].count()}")

Before cleaning empty hotel name: 325
Before cleaning empty hotel name: 300


In [13]:
clean_hotel_df

# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather 
#description and maximum temperature for the city.

info_box_template = """
<dl>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure

# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
#locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))